In [1]:
import benchmark
from memory import memset_zero
from random import rand

In [2]:
alias type = DType.float32

struct Matrix[rows: Int, cols: Int]:
    var data: DTypePointer[type]

    # Initialize zeroeing all values
    fn __init__(inout self):
        self.data = DTypePointer[type].alloc(rows * cols)
        memset_zero(self.data, rows * cols)

    # Initialize taking a pointer, don't set any elements
    fn __init__(inout self, data: DTypePointer[type]):
        self.data = data

    # Initialize with random values
    @staticmethod
    fn rand() -> Self:
        var data = DTypePointer[type].alloc(rows * cols)
        rand(data, rows * cols)
        return Self(data)

    fn __getitem__(self, y: Int, x: Int) -> Scalar[type]:
        return self.load[1](y, x)

    fn __setitem__(self, y: Int, x: Int, val: Scalar[type]):
        self.store[1](y, x, val)

    fn load[nelts: Int](self, y: Int, x: Int) -> SIMD[type, nelts]:
        return self.data.load[width=nelts](y * self.cols + x)

    fn store[nelts: Int](self, y: Int, x: Int, val: SIMD[type, nelts]):
        return self.data.store[width=nelts](y * self.cols + x, val)
    fn __del__(owned self):
        self.data.free()
    fn __copyinit__(inout self, existing: Self):
        self.data = DTypePointer[type].alloc(rows * cols)
        memcpy[rows * cols](self.data, existing.data)
        

In [3]:
import math

In [24]:

struct Column[rows: Int]:
    alias Col = Matrix[rows, 1]
    var elements: Self.Col
    fn __init__(inout self):
        self.elements.__init__()
    fn __init__(inout self, matrix: Matrix[rows, 1]):
        self.elements = matrix
    @staticmethod
    fn rand() -> Self:
        return Self.Col.rand()
    fn __getitem__(self, i: Int) raises -> Scalar[type]:
        if i >= rows:
            # note that DTypePointer has no bounds checking
            raise Error("index out of range")
        return self.elements[i, 0]
    fn __setitem__(inout self, i: Int, val: Scalar[type]):
        self.elements[i, 0] = val
    fn __copyinit__(inout self, existing: Self):
        self.elements = existing.elements
    fn softmax(self) raises -> Self:
        var x_max: Scalar[type]  = 0
        for i in range(rows):
            var x = self[i]
            if x > x_max:
                x_max = x
        var d: Scalar[type] = 0
        for i in range(rows):
            var x = self[i]
            d += math.exp(x - x_max)
        var probs = Self()
        for i in range(rows):
            var x = self[i]
            probs[i] = math.exp(x - x_max) / d
        return probs 



In [27]:
var logits = Column[10].rand()
print(logits[0], logits[1], logits[2], logits[3])
var probs = logits.softmax()
print(probs[0], probs[1], probs[2], probs[3])
var s: Scalar[type] = 0.0
for i in range(10):
    s += probs[i]
print("sum of probability", s, end="\n")

0.62686139345169067 0.7093198299407959 0.2802889347076416 0.37190967798233032
0.12561589479446411 0.13641302287578583 0.088823951780796051 0.097346529364585876
sum of probability 0.99999994039535522
